<a href="https://colab.research.google.com/github/samyukthagopalsamy/PROJECT-WORK-II/blob/main/ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required libraries**

In [2]:
from google.colab import files 
import pandas as pd 
import io 

In [3]:
uploaded = files.upload()

Saving checkpoints_coordinates_distance.csv to checkpoints_coordinates_distance.csv


In [4]:
distance_time_speed_dataset = pd.read_csv(io.BytesIO(uploaded['checkpoints_coordinates_distance.csv']), index_col=0, low_memory=False) 
print(distance_time_speed_dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131581 entries, 0 to 131580
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tracking_number           131581 non-null  int64  
 1   source_city               131581 non-null  object 
 2   destination_city          131581 non-null  object 
 3   carrier                   131581 non-null  object 
 4   time_to_delivery_hours    131581 non-null  float64
 5   distance                  81987 non-null   float64
 6   source_country_code       93233 non-null   object 
 7   destination_country_code  103117 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 9.0+ MB
None


In [5]:
distance_time_speed_dataset = distance_time_speed_dataset[distance_time_speed_dataset.time_to_delivery_hours != 0]

In [6]:
distance_time_speed_dataset = distance_time_speed_dataset[distance_time_speed_dataset['distance'].notnull()]
distance_time_speed_dataset.reset_index(drop=True, inplace=True)

**Calculating speed**

In [7]:
distance_time_speed_dataset['speed'] = distance_time_speed_dataset.distance / (distance_time_speed_dataset.time_to_delivery_hours)
distance_time_speed_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81986 entries, 0 to 81985
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tracking_number           81986 non-null  int64  
 1   source_city               81986 non-null  object 
 2   destination_city          81986 non-null  object 
 3   carrier                   81986 non-null  object 
 4   time_to_delivery_hours    81986 non-null  float64
 5   distance                  81986 non-null  float64
 6   source_country_code       81986 non-null  object 
 7   destination_country_code  81986 non-null  object 
 8   speed                     81986 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 5.6+ MB


**Carrier distribution**

In [8]:
from collections import Counter
from tabulate import tabulate
distribution = []
carrier = distance_time_speed_dataset['carrier']
counter = Counter(carrier)
for key, value in counter.items():
  percent = value / len(carrier) * 100
  df = {'Carrier':key, 'Count of unique values':value, 'Percentage':percent}
  distribution.append(df)
dist = pd.DataFrame(distribution) 
dist = dist.sort_values(by=['Carrier'], ignore_index=True)

In [9]:
group = distance_time_speed_dataset.groupby('carrier')
df1 = group.apply(lambda x: len(x['source_city'].unique()))
df2 = group.apply(lambda x: len(x['destination_city'].unique()))
df3 = group.apply(lambda x: sum(x['time_to_delivery_hours'])/len(x['time_to_delivery_hours']))
df4 = group.apply(lambda x: sum(x['speed'])/len(x['speed']))
df5 = group.apply(lambda x: sum(x['distance'])/len(x['distance']))

dist['Unique source cities'] = df1.values
dist['Unique destination cities'] = df2.values
dist['Average delivery time'] = df3.values
dist['Average speed'] = df4.values
dist['Average distance'] = df5.values

In [10]:
dist

,Carrier,Count of unique values,Percentage,Unique source cities,Unique destination cities,Average delivery time,Average speed,Average distance
0,aramex,413,0.503745,63,99,154.239709,36.942388,5021.693000
1,bluedart,42794,52.196717,418,1635,83.713207,99.526230,4409.540923
2,delhivery,4106,5.008172,225,1419,93.183390,13.895743,1117.931932
3,dhl_express,2131,2.599224,84,216,76.090568,208.713372,12412.411243
4,dotzot,690,0.841607,16,152,147.324638,8.338129,1004.755641
5,ecom_express,1743,2.125973,72,719,132.888698,13.983192,1542.309086
6,fedex,25941,31.640768,110,752,91.379669,16.877349,1195.317723
7,xpressbees,4168,5.083795,202,604,120.800384,5.570874,630.849618


In [11]:
one_hot_encoded_training_predictors = pd.get_dummies(distance_time_speed_dataset['carrier'])
distance_time_speed_dataset = distance_time_speed_dataset.join(one_hot_encoded_training_predictors)
del distance_time_speed_dataset['carrier']

In [12]:
distance_time_speed_dataset.sample()

,tracking_number,source_city,destination_city,time_to_delivery_hours,distance,source_country_code,destination_country_code,speed,aramex,bluedart,delhivery,dhl_express,dotzot,ecom_express,fedex,xpressbees
58718,59649290702,cod processing,jabalpur rural,45.0,12229.896587,us,in,271.77548,0,1,0,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(distance_time_speed_dataset, test_size=0.2, random_state=42, shuffle=True)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [14]:
X = train[['speed', 'distance', 'aramex', 'bluedart',	'delhivery',	'dhl_express',	'dotzot',	'ecom_express',	'fedex',	'xpressbees']]
Y = train['time_to_delivery_hours']
Xt = test[['speed', 'distance', 'aramex', 'bluedart',	'delhivery',	'dhl_express',	'dotzot',	'ecom_express',	'fedex',	'xpressbees']]
result = pd.DataFrame()
result['Actual'] = test['time_to_delivery_hours']

# standardizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
Xt = scaler.transform(Xt)
Y = Y.values.reshape((len(Y), 1)) 
Y = scaler.fit_transform(Y)
Y = Y.ravel()

**Gradient Boosting for regression.**

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor( loss = 'huber', learning_rate=0.001, n_estimators=350, max_depth=6 , subsample=1, verbose=False, random_state=126)  
gbr.fit(X, Y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.001, loss='huber',
                          max_depth=6, max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=350,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=126, subsample=1, tol=0.0001,
                          validation_fraction=0.1, verbose=False,
                          warm_start=False)

In [20]:
y_predicted_gbr = scaler.inverse_transform(gbr.predict(Xt))
result['predicted_gbr'] = y_predicted_gbr

In [21]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(result["Actual"], result["predicted_gbr"])
mse

6472.870633877866

In [29]:
X.shape

(65588, 10)

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [30]:
def larger_model():
    model = Sequential()
    model.add(Dense(20, input_dim=10, kernel_initializer='normal', activation='relu'))
    #model.add(Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(Dense(7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [31]:
model = larger_model()
model.fit(X,Y,epochs=200, batch_size=20)

Epoch 1/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.5395
Epoch 2/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.3884
Epoch 3/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.2763
Epoch 4/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.2603
Epoch 5/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.4223
Epoch 6/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.2583
Epoch 7/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.3529
Epoch 8/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.2691
Epoch 9/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.3318
Epoch 10/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.2351
Epoch 11/200
3280/3280 [==============================] - 4s 1ms/step - loss: 0.2235
Epoch 12/200
3280/3280 [==============================] - 4s 1ms/step - lo

In [32]:
y_predicted_sequential = scaler.inverse_transform(model.predict(Xt))
result['predicted_sequential'] = y_predicted_sequential
mse = mean_squared_error(result["Actual"], result["predicted_sequential"])
mse

1532.924882646717

In [ ]:
# from sklearn.model_selection import StratifiedShuffleSplit
# X = distance_time_speed_dataset[['speed', 'distance', 'carrier']]
# y = distance_time_speed_dataset['time_to_delivery_hours']
# sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
# sss.get_n_splits(X, y)
# sss.split(X, y)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier 
# scores = [] 

# # using regression to get predicted data 
# rf = RandomForestClassifier(n_estimators=40, max_depth=7) 
# for train_index, test_index in sss.split(X, y): 
# 	X_train, X_test = X[train_index], X[test_index] 
# 	y_train, y_test = y[train_index], y[test_index] 
# 	rf.fit(X_train, y_train) 
# 	pred = rf.predict(X_test) 
# 	scores.append(accuracy_score(y_test, pred)) 

# # get accurracy of each prediction 
# print(scores) 
